Импортируем библиотеки и читаем данные

In [ ]:
import pandas as pd

data = pd.read_csv('googleplaystore.csv')

Сохраним в переменные `data_head` и `data_tail` первые и последние 3 строки из датафрейма соответственно

In [ ]:
data_head = data.head(3)
data_tail = data.tail(3)

In [ ]:
data_head

In [ ]:
data_tail

Сохраним в переменные `n_col` и `n_row` количество столбцов и строк в датафрейме соответственно

In [ ]:
n_row, n_col = data.shape

In [ ]:
n_row, n_col

Посмотрим, какое количество уникальных приложений есть в наших данных (колонка **App**).

In [ ]:
data['App'].nunique()

Посчитаем число пропущенных значений в колонке с рейтингом приложений. Сохраним в переменную `rating_missing` количество пропущенных значений в колонке **Rating**. 

In [ ]:
rating_missing = data['Rating'].isnull().sum()

In [ ]:
rating_missing

Соберем датафрейм, который будет включать в себя: 
- первые три строки исходного датафрейма, 
- строки 6-8 (включительно), 
- строки 16-19 (включительно) 
- и колонки: **App, Size, Genres, Current Ver** в соответствующем порядке.

По сути, это будет выглядеть как объединение трех, состоящих из нескольких строк, датафреймов, являющимися срезами из исходных данных.
Сохраним полученный датафрейм в формате csv.


In [ ]:
data_1_3 = data[['App', 'Size', 'Genres', 'Current Ver']][:3]
data_6_8 = data[['App', 'Size', 'Genres', 'Current Ver']][6:9]
data_16_19 = data[['App', 'Size', 'Genres', 'Current Ver']][16:20]
result = pd.concat([data_1_3, data_6_8, data_16_19])
result.to_csv('output/concatenated_data.csv', sep=';')

In [ ]:
pd.read_csv('output/concatenated_data.csv', sep=';', index_col=0)

Для дальнейшего анализа нам необходимо убрать дубликаты приложений (колонка **App**) и сбросить индекс. Сохраним полученный результат в виде датафрейма в переменную `unique_playstore`. 

In [ ]:
unique_playstore = data.drop_duplicates(subset='App').reset_index(drop=True)

Приведем названия к стандартному виду — все буквы должны быть нижнего регистра, а пробелы должны быть заменены на нижние подчеркивания. 

In [ ]:
playstore = unique_playstore.copy()
playstore.columns = playstore.columns.str.lower().str.replace(' ', '_')
# unique_playstore.rename(lambda column: column.lower().replace(' ', '_'), 
#                         axis='columns', inplace=True)

In [ ]:
playstore.head()

Посмотрим на долю платных и бесплатных приложений. Сохраним долю бесплатных приложений в переменной `free_app_share`, округлив до сотых. В качестве десятичного разделителя используем точку.

In [ ]:
playstore.value_counts(subset='type', normalize=True)

In [ ]:
free_app_share = round(playstore.value_counts(subset='type', normalize=True)['Free'], 2)

Отберем из датафрейма только те приложения, которые относятся к образовательной категории  (**EDUCATION**) и с количеством отзывов пользователей более 1000. Сохраним новый датафрейм в переменную `education_playstore`.

In [ ]:
education_playstore = playstore[(playstore['reviews'] > 1000) 
                                & (playstore['category'] == 'EDUCATION')]     
education_playstore.reset_index(drop=True, inplace=True)       

In [ ]:
education_playstore.head()

Уберем лишние символы из колонки с ценой (**price**), чтобы далее с ней было удобнее работать, и переведем данные в тип `float`.

In [ ]:
playstore['price'] = playstore['price'].apply(lambda x: x.replace("$", "")).astype(float)

Сделаем сводную таблицу по каждой категории со средним значением цены и рейтинга для платных и бесплатных приложений. Результат сохраним в формате csv.

In [ ]:
pivot_table = pd.pivot_table(playstore, index=['category', 'type'],
                             values=['price', 'rating'], aggfunc='mean')
pivot_table.rename(columns={'price': 'mean_price',
                   'rating': 'mean_rating'}, inplace=True)
pivot_table[['mean_price', 'mean_rating']] = pivot_table[[
    'mean_price', 'mean_rating']].round(2)
pivot_table.to_csv('output/pivot_table.csv', sep=',')
pivot_table

In [ ]:
pd.read_csv('output/pivot_table.csv', sep=',')